Ever had a gazillion tabs open and didn't know what was what and wanted to manage / clean things up?
Well this code could be for you!
Buggy as the code removes from the recovery file and leaves blank tabs in it's place.
And firefox doesn't like being repeatedly shut down perhaps and a message about starting up in safe mode pops up.
Just close the window and run the bat file again (see end of this notebook)

Scroll down to the user settings part of the file to set things up.
Reading through the comments should give a fair idea of what does what

This contains AI generated code, prompted, arranged, and tweaked by me (https://github.com/jasonmhead / https://jasonmhead.com)

In [ ]:
#!pip install lz4
import os
import shutil
import lz4.block
import random
import json
import subprocess

In [ ]:
def extract_firefox_info(file_path):
    """
    Extracts Firefox information from a recovery file.

    Args:
        file_path (str): The path to the Firefox recovery file.

    Returns:
        dict: Extracted information as an object.

    Raises:
        FileNotFoundError: If the file is not found.
    """
    try:
        # Open the recovery file in binary mode
        with open(file_path, "rb") as file:
            # Read the entire content of the file
            compressed_data = file.read()

            # Skip past the header by slicing the compressed data
            decompressed_data = lz4.block.decompress(compressed_data[8:])

            # Decode the decompressed data as UTF-8 and load it as JSON
            json_data = json.loads(decompressed_data.decode("utf-8"))

            # Create a dictionary to store the extracted information
            extracted_info = {"windows": []}

            # Iterate through the windows, tabs, and page entries to extract URLs
            for entry in json_data["windows"]:
                window_info = {"tabs": []}
                for tab in entry["tabs"]:
                    tab_info = {}
                    if "entries" in tab:
                        tab_info["entries"] = []
                        for page in tab["entries"]:
                            tab_info["entries"].append({"url": page["url"]})
                    window_info["tabs"].append(tab_info)
                extracted_info["windows"].append(window_info)

            # Return the extracted information as an object
            return extracted_info

    except FileNotFoundError:
        return {"error": "File not found."}
        return "File not found."
    

    
def list_urls(data):
    """
    Lists out the URLs from the extracted Firefox information.

    Args:
        data (dict): Extracted information as an object.

    Returns:
        str: Numbered list of URLs.
    """
    url_list = ""
    counter = 1

    url_list = {}
    for window in data["windows"]:
        for tab in window["tabs"]:
            if "entries" in tab:
                for page in tab["entries"]:
                    url_list[counter] = page['url'][0:100]
                    counter += 1

    return url_list



def search_dictionary(dictionary, search_value):
    keys = [str(key) for key, value in dictionary.items() if search_value in value]
    keys = sorted(map(int, keys))  # Convert to integers and sort
    result = []
    start = None
    end = None
    for key in keys:
        if start is None:
            start = key
            end = key
        elif key == end + 1:
            end = key
        else:
            if start == end:
                result.append(str(start))
            else:
                result.append(f"{start}-{end}")
            start = key
            end = key
    if start is not None:
        if start == end:
            result.append(str(start))
        else:
            result.append(f"{start}-{end}")
    return ','.join(result)


def delete_entries(data, entry_numbers):
    """
    Deletes entries from the Firefox information based on the given entry numbers and ranges.

    Args:
        data (dict): Extracted information as an object.
        entry_numbers (str): Comma-separated string of entry numbers and ranges to delete.
        example: 1,5-10,20,23-56,200

    Returns:
        dict: Modified Firefox information after deleting entries.
    """
    entry_numbers = entry_numbers.split(",")  # Split the string into a list of entry numbers and ranges

    # Create a set to store the URLs to delete
    urls_to_delete = set()

    for entry in entry_numbers:
        if "-" in entry:
            start, end = entry.split("-")
            urls_to_delete.update(get_urls_in_range(data, int(start), int(end)))
        else:
            urls_to_delete.add(get_url_at_index(data, int(entry)))

    for window in data["windows"]:
        for tab in window["tabs"]:
            if "entries" in tab:
                tab["entries"] = [page for page in tab["entries"] if page["url"] not in urls_to_delete]

    return data


def get_url_at_index(data, index):
    """
    Returns the URL at the given index in the Firefox information.

    Args:
        data (dict): Extracted information as an object.
        index (int): Index of the URL to retrieve.

    Returns:
        str: URL at the given index.
    """
    urls = []

    for window in data["windows"]:
        for tab in window["tabs"]:
            if "entries" in tab:
                urls.extend([page["url"] for page in tab["entries"]])

    return urls[index - 1] if 0 <= index <= len(urls) else None


def get_urls_in_range(data, start, end):
    """
    Returns a set of URLs within the given range of indices in the Firefox information.

    Args:
        data (dict): Extracted information as an object.
        start (int): Start index of the range.
        end (int): End index of the range.

    Returns:
        set: URLs within the given range of indices.
    """
    urls = []

    for window in data["windows"]:
        for tab in window["tabs"]:
            if "entries" in tab:
                urls.extend([page["url"] for page in tab["entries"]])

    urls_range = urls[start - 1:end] if 0 <= start <= len(urls) and 0 <= end <= len(urls) else []
    return set(urls_range)



def write_modified_jsonlz4(file_path, modified_data):
    """
    Creates a copy of the given JSONLZ4
    Writes the modified data to a the JSONLZ4 file in Firefox format.

    Args:
        file_path (str): Path to the original JSONLZ4 file.
        modified_data (dict): Modified Firefox data to write.

    Returns:
        None
    """
    
    ran_num = round(random.uniform(0, 10000))
    
    # Create a copy of the original JSONLZ4 file in the same directory
    copy_file_path = os.path.splitext(file_path)[0] + "_copy_"+str(ran_num)+".jsonlz4"
    shutil.copy2(file_path, copy_file_path)

    # Encode the modified data as JSON
    modified_json = json.dumps(modified_data).encode("utf-8")

    # Compress the modified data
    compressed_data = lz4.block.compress(modified_json)

    # Add the proper header to the compressed data
    header = b"mozLz40\0"
    output_data = header + compressed_data

    # Write the modified data to the new JSONLZ4 file
    with open(file_path, "wb") as modified_file:
        modified_file.write(output_data)
        
def run_batch_file(file_path):
    try:
        # Run the batch file using subprocess
        subprocess.run(file_path, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while executing the batch file: {e}")


<h1>User Settings Here</h1>

In [ ]:
user_name = "your username"
session_folder = "firefox session folder"
backups_location = r'C:\Users\\' + user_name + r'\AppData\Roaming\Mozilla\Firefox\Profiles\\'+session_folder+r'\sessionstore-backups\\'

recovery_file = "recovery.jsonlz4"
recovery_backup_file = "recovery.baklz4"
previous_file = "previous.jsonlz4"

session_files = [recovery_file, recovery_backup_file, previous_file]

file_path = backups_location + recovery_file

In [ ]:
file_path

In [ ]:
# parse the session file
firefox_info = extract_firefox_info(file_path)
print(firefox_info)

In [ ]:
# get a numbered list of urls
numbered_urls = list_urls(firefox_info)
numbered_urls

In [ ]:
# search and match urls by substring
numbered_urls = list_urls(firefox_info)
url_search_result = search_dictionary(numbered_urls, "search text")
print(url_search_result) 

In [ ]:
# Delete entries
entries_to_delete = url_search_result  # Example entry numbers to delete as a comma-separated string

if(entries_to_delete):
    modified_data = delete_entries(firefox_info, entries_to_delete)
    firefox_info = modified_data # make sure that a delete can't be repeated twice
    print(modified_data)


In [ ]:
# to close empty tabs
# I didn't have any success with this for this purpose, but the concept is pretty cool
# perhaps the url data isn't being properly removed from the session file and so leaves blank tabs
# https://addons.mozilla.org/en-US/firefox/addon/close-unused-blank-tabs/

In [ ]:
    # make sure you really want to remove the matched urls from these files
    # copies are mode for resoration just in case

    #for path in session_files:
    #    file_path = backups_location + recovery_file
    # todo: tie all functions together to loop through session files

write_modified_jsonlz4(file_path, modified_data)

In [ ]:
# this file is in the repo as well
run_batch_file(r"path-to/close_and_reopen_firefox.bat")